In [20]:
import threading
import time

class BakeryAlgorithm:
    def __init__(self, num_processes):
        self.num_processes = num_processes
        self.choosing = [False] * num_processes
        self.tickets = [0] * num_processes

    def lock(self, process_id):
        self.choosing[process_id] = True
        self.tickets[process_id] = max(self.tickets) + 1
        self.choosing[process_id] = False

        for i in range(self.num_processes):
            if i == process_id:
                continue
            while self.choosing[i]:
                pass
            while self.tickets[i] != 0 and (
                (self.tickets[i] < self.tickets[process_id]) or
                (self.tickets[i] == self.tickets[process_id] and i < process_id)
            ):
                pass

    def unlock(self, process_id):
        self.tickets[process_id] = 0


def compute_partial_factorial(start, end, process_id, bakery, shared_resource):
    """Compute the product from `start` to `end`."""
    bakery.lock(process_id)

    partial_result = 1
    for i in range(start, end + 1):
        partial_result *= i
    shared_resource[process_id] = partial_result

    bakery.unlock(process_id)


def compute_sequential_factorial(number):
    """Compute the factorial sequentially."""
    result = 1
    for i in range(1, number + 1):
        result *= i
    return result


if __name__ == "__main__":
    number = 1550  # The number to compute the factorial of
    num_threads = 5
    step = number // num_threads  # Split the range equally among threads

    # Sequential execution
    start_time = time.time()
    sequential_result = compute_sequential_factorial(number)
    sequential_time = time.time() - start_time
    print(f"Sequential factorial result: {sequential_result}")
    print(f"Time taken for sequential execution: {sequential_time:.6f} seconds\n")

    # Thread-based execution
    shared_resource = [1] * num_threads  # Store partial results
    bakery = BakeryAlgorithm(num_threads)

    threads = []
    start_time = time.time()
    for i in range(num_threads):
        start = i * step + 1
        end = (i + 1) * step if i < num_threads - 1 else number
        t = threading.Thread(target=compute_partial_factorial, args=(start, end, i, bakery, shared_resource))
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    # Compute the final factorial by multiplying all partial results
    thread_result = 1
    for partial_result in shared_resource:
        thread_result *= partial_result

    thread_time = time.time() - start_time
    print(f"Thread-based factorial result: {thread_result}")
    print(f"Time taken for thread-based execution: {thread_time:.6f} seconds\n")

    # Comparison
    print(f"Performance comparison:")
    print(f"Sequential time: {sequential_time:.6f} seconds")
    print(f"Thread-based time: {thread_time:.6f} seconds")
    if sequential_time < thread_time:
        print("Sequential execution was faster.")
    else:
        print("Thread-based execution was faster.")


Sequential factorial result: 71115512856145236107664967727509046083193211653746748227098196542702845042801663156155926767098383136903633410128548087816659268210019764379430822801067441182801974417686068672297459189102730492736806331177373413636267487121832869184819574693514981732982972086191997273440805744741435737193147897878436023652769541205727798863913681401612887685246034390637736414576347957803650828118469317748664373369810485493345594683285461970748285452406476906780891183402674517913851109762470188667681330581646826341882109562198809239985904433589439291672239086875276203178882266809081478000242560253807435221547868144508880911466892540557827383299196703602993400331983261750566101548959075652862685726390695465232741747580133636018797346856397484507299061625318836980751018579185452127381249656591187083315835866739461103381028314091408426587129603835294248383354539188398160696085140476351416544522676520970096565397190748361103993955646431378280227309839027808436685387268120818863